In [1]:
import Base: reset

In [3]:
using Distributions, LightGraphs, ProgressMeter, RCall, DataFrames

In [4]:
srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.85292, 1.34669, 1.51821, 1.75166, 1.45124, 1.02233, 1.94478, 1.93202, 1.84752, 1.83554  …  1.3986, 1.70216, 1.46308, 1.27622, 1.57881, 1.51321, 1.79293, 1.26551, 1.60442, 1.31848], 382)

The grammar is a set of simple building blocks that generalize any diffusion simulation. 

 > `reset(N) -> seed(node_status, N, seed_fn, seed_size) -> 
            evolve(node_status, N, evolve_fn, ...) -> 
            sum(node_status)`  

The data structure on which diffusion simulations are executed is a combination of the network connections between the nodes and their *immutable* properties. All of them are initialized together at the beginning of each realization and is not tampered with in the course of the simulation. This data structure might hold different parameters for different kinds of models, for e.g., it might hold thresholds or influence of neighbors or influence of advertising. We illustrate our design using a simple simulation involving thresholds.  

In [5]:
struct Network
    G::LightGraphs.SimpleGraphs.SimpleGraph{Int}
    threshold::Vector{Float64}
end

### 1. Initialization

The starting point of any diffusion simulation is the `reset` function that takes a network object and sets the status of all the nodes in the network to `false`. The output is a BitVector.

In [6]:
function reset(N::Network)
    return falses(nv(N.G))
end

reset (generic function with 4 methods)

### 2. Seeding

The next step is to seed a subset of the network using a seeding function that dictates which specific nodes should be targeted. This mutates the state of the `node_status` vector in place. A purer version of this function would initialize a new vector that is the copy of `node_status`, update this vector and return a copy of the new vector. Let us define these two versions and check if there are any speed differences between the versions.

In [7]:
function seed!(node_status::BitVector, 
               N::Network,
               seed_fn::Function,
               seed_size::Int)
    
    seed = seed_fn(N, seed_size) # should return a vector of indices
    node_status[seed] = true
    
end

seed! (generic function with 1 method)

In [8]:
function seed(node_status::BitVector,
              N::Network,
              seed_fn::Function,
              seed_size::Int)
    
    new_status = copy(node_status)
    seed = seed_fn(N, seed_size) # should return a vector of indices
    new_status[seed] = true
    
    return new_status
end

seed (generic function with 1 method)

We illustrate the working of these functions and measure the timing using two examples. 

#### 2.1 Random seeding

In [9]:
function seed_random(N::Network, seed_size::Int)
    return sample(vertices(N.G), seed_size, replace = false)
end

seed_random (generic function with 1 method)

In [10]:
N = Network(watts_strogatz(10^4, 3, 0.5), rand(TruncatedNormal(0.05, 0.05, 0, Inf), 10^4))

Network({10000, 10000} undirected simple Int64 graph, [0.084108, 0.0325621, 0.0378909, 0.0202239, 0.122579, 0.0759791, 0.132814, 0.107071, 0.00187806, 0.00291333  …  0.0861788, 0.0601747, 0.0314736, 0.0978747, 0.0534731, 0.0600227, 0.0795918, 0.117065, 0.0288046, 0.0631423])

In [12]:
node_status = reset(N);

In [13]:
@time seed!(node_status, N, seed_random, 1000)

  0.304894 seconds (113.94 k allocations: 6.082 MiB)


true

In [14]:
sum(node_status)

1000

In [15]:
node_status = reset(N);

In [16]:
@time node_status = seed(node_status, N, seed_random, 1000);

  0.033944 seconds (5.62 k allocations: 387.856 KiB)


In [17]:
sum(node_status)

1000

#### 2.2 Seeding the nodes with highest pagerank centrality 

In [18]:
function seed_pagerank(N::Network, seed_size::Int)
    return sortperm(pagerank(N.G))[1:seed_size]
end

seed_pagerank (generic function with 1 method)

In [19]:
node_status = reset(N)
@time seed!(node_status, N, seed_pagerank, 1000)
sum(node_status)

  0.277631 seconds (111.82 k allocations: 6.251 MiB, 3.03% gc time)


1000

In [20]:
node_status = reset(N)
@time node_status = seed(node_status, N, seed_pagerank, 1000)
sum(node_status)

  0.027593 seconds (1.26 k allocations: 379.803 KiB)


1000

This shows an interesting pattern when the bottleneck is shifted to the seeder algorithm. The more complex the seeding algorithm, lesser is the difference in speed between mutating and non-mutating updates.

By keeping the `seeder` function is independent of the `seed` function, we can quickly compare the relative benefits of the diffusion process across seeding methods.

### 3. Evolution

Different simulation design will employ different kinds of rules for node behavior. These rules are encoded into the `evolve_fn`. The `evolve_fn` function might be passed additional variables from its calller to support the evolution logic.

In [21]:
function evolve!(node_status::BitVector, 
                 N::Network,
                 evolve_fn::Function)
    
   for node in shuffle(vertices(N.G)) # by default nodes are updated in random order
        if node_status[node] == false
            evolve_fn(node_status, N, node)
        end
    end
end

evolve! (generic function with 1 method)

This higher level abstraction allows us to specify the evolution separate from the mechanics of evolving the diffusion process. The type and number of inputs to the evolver function can change depending on the complexity of the evolution rule.

We illustrate how the evolution works using a few example functions.

In [22]:
function evolve_threshold!(node_status::BitVector, N::Network, node::Int)
    
    n_engaged_nbrs = sum([node_status[nbr] for nbr in neighbors(N.G, node)])
    
    if n_engaged_nbrs/nv(N.G) > N.threshold[node] 
        node_status[node] = true
    end
end

evolve_threshold! (generic function with 1 method)

In [33]:
@time evolve!(node_status, N, evolve_threshold!)

  0.003651 seconds (27.00 k allocations: 1.312 MiB)


In [34]:
sum(node_status)

1002

The `evolve` function is typically called multiple times in a typical simulation design.

### 4. Executing the simulation

The vocabulary of functions defined above is called several times from the function `simulate` that is incharge of putting together the parameter space required to execute the simulations. The function should collect the results into a relational database.

As an example illustrating the entire process, we illustrate the simulation of the following problem:

*Given a network, seeding individuals with high pagerank provides more bang for the buck compared to random seeding*



Let us consider Watts-Strogatz networks with $10,000$ nodes and with each individual having an average degree of $1 - 10$. The threshold for each node can vary from $0.05 - 0.3$ (but we assume that everyone has the same threshold). We consider ten values of threshold in between these values.

In [35]:
function simulate(n::Int, p::Float64; T = 30, n_realizations = 20)
    parameter_space = [(k, phi) for k in 1:10, 
                                    phi in linspace(0.05, 0.3, 10)]
    
    output = DataFrame(r = Int[], k = Int[], phi = Float64[], engaged_random = Int[], engaged_pagerank = Int[])
    
    @showprogress 1 "Simulating..." for (k, phi) in parameter_space
          for r in 1:n_realizations
            N = Network(watts_strogatz(n, k, p), fill(phi, n))
            
            # Begin the diffusion process starting with a random seed
            
            node_status = reset(N)
            
            seed!(node_status, N, seed_random, 1) # start with a single random seed
            
            for t = 1:T
                evolve!(node_status, N, evolve_threshold!)
            end
            
            n_engaged_rand = sum(node_status)
            
            # Begin the diffusion process starting with purposive seeding
            
            node_status = reset(N)
            
            seed!(node_status, N, seed_pagerank, 1)
            
            for t = 1:T
                evolve!(node_status, N, evolve_threshold!)
            end
            
            n_engaged_pagerank = sum(node_status)
            
            push!(output, [r, k, phi, n_engaged_rand, n_engaged_pagerank])
            
        end
    end
    
    return output
    
end

simulate (generic function with 1 method)

In [36]:
results = simulate(10^4, 0.1)

Simulating...100%|██████████████████████████████████████| Time: 0:04:45


,r,k,phi,engaged_random,engaged_pagerank
1,1,1,0.05,1,1
2,2,1,0.05,1,1
3,3,1,0.05,1,1
4,4,1,0.05,1,1
5,5,1,0.05,1,1
6,6,1,0.05,1,1
7,7,1,0.05,1,1
8,8,1,0.05,1,1
9,9,1,0.05,1,1
10,10,1,0.05,1,1


In [37]:
@rput results

,r,k,phi,engaged_random,engaged_pagerank
1,1,1,0.05,1,1
2,2,1,0.05,1,1
3,3,1,0.05,1,1
4,4,1,0.05,1,1
5,5,1,0.05,1,1
6,6,1,0.05,1,1
7,7,1,0.05,1,1
8,8,1,0.05,1,1
9,9,1,0.05,1,1
10,10,1,0.05,1,1


In [38]:
R"""
library(tidyverse)

"""

v ggplot2 2.2.1     v purrr   0.2.4
v tibble  1.4.2     v dplyr   0.7.4
v tidyr   0.8.0     v stringr 1.2.0
v readr   1.1.1     v forcats 0.2.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


RCall.RObject{RCall.StrSxp}
 [1] "forcats"   "stringr"   "dplyr"     "purrr"     "readr"     "tidyr"    
 [7] "tibble"    "ggplot2"   "tidyverse" "stats"     "graphics"  "grDevices"
[13] "utils"     "datasets"  "methods"   "base"     


In [39]:
R"""
summary_results <- results %>% group_by(k, phi) %>% 
                              summarize_all(funs(mean)) %>% 
                              select(-r)
"""

RCall.RObject{RCall.VecSxp}
# A tibble: 100 x 4
# Groups:   k [10]
       k    phi engaged_random engaged_pagerank
   <int>  <dbl>          <dbl>            <dbl>
 1     1 0.0500           1.00             1.00
 2     1 0.0778           1.00             1.00
 3     1 0.106            1.00             1.00
 4     1 0.133            1.00             1.00
 5     1 0.161            1.00             1.00
 6     1 0.189            1.00             1.00
 7     1 0.217            1.00             1.00
 8     1 0.244            1.00             1.00
 9     1 0.272            1.00             1.00
10     1 0.300            1.00             1.00
# ... with 90 more rows


In [40]:
R"""
summary(summary_results)
"""

RCall.RObject{RCall.StrSxp}
       k             phi         engaged_random engaged_pagerank
 Min.   : 1.0   Min.   :0.0500   Min.   :1      Min.   :1       
 1st Qu.: 3.0   1st Qu.:0.1056   1st Qu.:1      1st Qu.:1       
 Median : 5.5   Median :0.1750   Median :1      Median :1       
 Mean   : 5.5   Mean   :0.1750   Mean   :1      Mean   :1       
 3rd Qu.: 8.0   3rd Qu.:0.2444   3rd Qu.:1      3rd Qu.:1       
 Max.   :10.0   Max.   :0.3000   Max.   :1      Max.   :1       
